# Using the FMI time series interface

https://github.com/fmidev/smartmet-plugin-timeseries/blob/master/docs/Using-the-Timeseries-API.md

https://github.com/fmidev/smartmet-plugin-timeseries/blob/master/docs/Examples.md

https://opendata.fmi.fi/timeseries?producer=airquality_urban&area=Helsinki&param=name,time,fmisid,stationname,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg

https://opendata.fmi.fi/timeseries?format=ascii&groupareas=0&separator=,&attributes=time,fmisid&producer=airquality_urban&area=Helsinki&param=time,fmisid,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg


In [27]:
import requests
import datetime
import pandas as pd
import numpy as np

In [51]:
end_time = datetime.datetime.utcnow()
start_time = end_time - datetime.timedelta(days=1)

fields = [
    'time',
    'fmisid',
    # 'stationname', 
    'PM10_PT1H_avg',
    'PM25_PT1H_avg', 
    'O3_PT1H_avg',
    'CO_PT1H_avg',
    'SO2_PT1H_avg', 
    'NO2_PT1H_avg',
    'TRSC_PT1H_avg',
]

url = 'https://opendata.fmi.fi/timeseries'

params = {
    'format': 'json',
    'precision': 'double',
    'groupareas': '0',
    'producer': 'airquality_urban',
    'area': 'Uusimaa',
    'param': ','.join(fields),
    'starttime': start_time.isoformat(timespec="seconds"),
    'endtime': end_time.isoformat(timespec="seconds"),
    'tz': 'UTC',
}

data = requests.get(url, params=params).json()

In [52]:
df = pd.DataFrame(data).astype({
    'fmisid': np.int32,
    'time': np.datetime64,
    'PM10_PT1H_avg': np.float64,
    'PM25_PT1H_avg': np.float64,
    'O3_PT1H_avg': np.float64,
    'CO_PT1H_avg': np.float64,
    'SO2_PT1H_avg': np.float64,
    'NO2_PT1H_avg': np.float64,
    'TRSC_PT1H_avg': np.float64,
})
#df = df.set_index(['fmisid', 'time'])
df[0:10]

,time,fmisid,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg
0,2022-08-24 20:00:00,100662,2.9,5.6,35.7,NaN,0.8,3.5,NaN
1,2022-08-24 21:00:00,100662,3.5,5.7,32.1,NaN,0.7,2.3,NaN
2,2022-08-24 22:00:00,100662,1.1,4.2,35.1,NaN,0.7,2.0,NaN
3,2022-08-24 23:00:00,100662,-3.5,2.9,39.2,NaN,0.7,1.8,NaN
4,2022-08-25 00:00:00,100662,-4.1,1.4,42.6,NaN,0.6,1.1,NaN
5,2022-08-25 01:00:00,100662,-2.2,1.0,42.2,NaN,0.6,0.9,NaN
6,2022-08-25 02:00:00,100662,-1.3,1.1,43.2,NaN,0.5,1.6,NaN
7,2022-08-25 03:00:00,100662,-1.1,0.6,41.7,NaN,0.5,3.5,NaN
8,2022-08-25 04:00:00,100662,-2.5,1.0,47.1,NaN,0.5,4.9,NaN
9,2022-08-25 05:00:00,100662,0.4,0.7,43.6,NaN,0.9,10.7,NaN


# DuckDB

In [53]:
import duckdb
con = duckdb.connect(database=':memory:')

In [54]:
con.execute('CREATE OR REPLACE TABLE airquality_urban AS SELECT * FROM df')

In [62]:
df2 = con.execute('SELECT * FROM airquality_urban').fetchdf()
df2.sample(10)

,time,fmisid,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg
100,2022-08-25 00:00:00,100762,2.5,1.8,38.2,NaN,NaN,1.5,NaN
262,2022-08-24 21:00:00,104074,11.9,7.8,NaN,NaN,NaN,3.0,NaN
122,2022-08-24 22:00:00,100763,4.6,3.9,NaN,NaN,NaN,1.5,NaN
375,2022-08-25 14:00:00,107400,11.1,NaN,NaN,NaN,NaN,10.2,NaN
331,2022-08-25 18:00:00,107147,9.8,5.1,NaN,NaN,NaN,14.1,NaN
155,2022-08-25 07:00:00,100803,5.5,2.8,52.9,NaN,NaN,3.5,NaN
124,2022-08-25 00:00:00,100763,-0.6,0.8,NaN,NaN,NaN,1.0,NaN
107,2022-08-25 07:00:00,100762,9.4,3.4,46.9,NaN,NaN,12.2,NaN
150,2022-08-25 02:00:00,100803,3.4,1.5,40.5,NaN,NaN,1.8,NaN
72,2022-08-24 20:00:00,100742,14.7,8.3,NaN,NaN,NaN,6.5,NaN
